In [1]:
from resnet_variant import *
from baseline_cnn import *
from resnet_variant import resnet_n
import torch.nn as nn
import torch.optim as optim
import time
import pathlib
import torch
from evaluation import Evaluation
from xray_imbalanced_dataloader import create_balanced_split_loaders
import random
from balanced_loss import W_CEL

%load_ext autoreload
%autoreload 2
# Setup: initialize the hyperparameters/variables
num_epochs = 1           # Number of full passes through the dataset
batch_size = 16          # Number of samples in each minibatch
learning_rate = 0.001  
seed = np.random.seed(1) # Seed the random number generator for reproducibility
p_val = 0.1              # Percent of the overall dataset to reserve for validation
p_test = 0.2             # Percent of the overall dataset to reserve for testing

class channelCopy(object):    
    def __call__(self, img):
        return torch.cat([img, img, img], 0)
    
def augmentImg(img): 
    if 0.9 < random.random():
        return img

    t=transforms.RandomRotation(10)
    img = t(img)
    return img
        
#TODO: Convert to Tensor - you can later add other transformations, such as Scaling here
transform = transforms.Compose([transforms.Resize(512),augmentImg,transforms.ToTensor(),channelCopy()])


# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

# Setup the training, validation, and testing dataloaders
# train_loader, val_loader, test_loader = create_split_loaders(batch_size, seed, transform=transform, 
#                                                              p_val=p_val, p_test=p_test,
#                                                              shuffle=True, show_sample=False, 
#                                                              extras=extras)
train_loader, val_loader, test_loader = create_balanced_split_loaders(batch_size, seed, transform=transform,
                                                                         p_val=p_val, p_test=p_test,
                                                                         shuffle=True, show_sample=False,
                                                                         extras=extras, z_score=True)
# Instantiate a BasicCNN to run on the GPU or CPU based on CUDA support
model = resnet_n(pretrained=False, num_classes=14)
model = model.to(computing_device)
print("Model on CUDA?", next(model.parameters()).is_cuda)

#TODO: Define the loss criterion and instantiate the gradient descent optimizer
#criterion = nn.MultiLabelSoftMarginLoss() #TODO - loss criteria are defined in the torch.nn package
# criterion = nn.MultiLabelSoftMarginLoss()
criterion = W_CEL()
#TODO: Instantiate the gradient descent optimizer - use Adam optimizer with default parameters
optimizer = optim.Adam(model.parameters()) #TODO - optimizers are defined in the torch.optim package

CUDA is supported
Model on CUDA? True


In [2]:
print(model)

ResNet_variant(
  (conv1): Conv2d(3, 64, kernel_size=(11, 11), stride=(2, 2), padding=(5, 5), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck_2(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(

In [3]:
# Set up folder for model saving
model_path = '{}/models/resnet_variant/{}/'.format(os.getcwd(), time.strftime("%Y%m%d-%H%M%S"))
model_pathlib = pathlib.Path(model_path)
if not model_pathlib.exists():
    pathlib.Path(model_pathlib).mkdir(parents=True, exist_ok=True)

#save loss in files for futher usage.
loss_path = '{}/losses/resnet_variant/{}/'.format(os.getcwd(), time.strftime("%Y%m%d-%H%M%S"))
loss_pathlib = pathlib.Path(loss_path)
if not loss_pathlib.exists():
    pathlib.Path(loss_pathlib).mkdir(parents=True, exist_ok=True)

In [ ]:
# Track the loss across training
total_loss = []
avg_minibatch_loss = []
loss_val_list = []
loss_val_min = float('inf')
N = 50
# Begin training procedure
for epoch in range(num_epochs):

    
    N_minibatch_loss = 0.0
    

    # Get the next minibatch of images, labels for training
    for minibatch_count, (images, labels) in enumerate(train_loader, 0):
#         if minibatch_count == 100:
#             break
        # Put the minibatch data in CUDA Tensors and run on the GPU if supported
        images, labels = images.to(computing_device), labels.to(computing_device)

        # Zero out the stored gradient (buffer) from the previous iteration
        optimizer.zero_grad()

        # Perform the forward pass through the network and compute the loss
        outputs = model(images)
        loss = criterion(outputs, labels)
#         print('training',minibatch_count,loss)
        # Automagically compute the gradients and backpropagate the loss through the network
        loss.backward()
        print(loss)
        # Update the weights
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss
        
        #TODO: Implement validation
        if minibatch_count % N == 49:
            #switch to evaluate mode
            model.eval()
            with torch.no_grad():
                loss_val = 0
                for count_val, (images_val, labels_val) in enumerate(val_loader):
#                     if count_val ==10:
#                         break
                    images_val, labels_val = images_val.to(computing_device), labels_val.to(computing_device)
                    outputs_val = model(images_val)
                    loss_val += criterion(outputs_val, labels_val)
#                     print('val',count_val, (loss_val/(count_val+1)))
                loss_val /= count_val
                print('val',minibatch_count,loss_val)
                loss_val_list.append(loss_val.item())
                if loss_val < loss_val_min:
                    model_name = "epoch_{}-batch_{}-loss_{}-{}.pt".format(epoch, minibatch_count, loss_val, time.strftime("%Y%m%d-%H%M%S"))
                    torch.save(model.state_dict(), os.path.join(model_path, model_name))
                    loss_val_min = loss_val
                    
        if minibatch_count % N == 0:    
            
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, minibatch_count, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss.item())
            N_minibatch_loss = 0.0
            
        prefix = 'resnet_variant_loss'
        with open(os.path.join(loss_path, prefix+"training.txt"), "w") as f:
            for s in total_loss:
                f.write(str(s) +"\n")

        with open(os.path.join(loss_path, prefix+"training_ave.txt"), "w") as f:
            for s in avg_minibatch_loss:
                f.write(str(s) +"\n")

        with open(os.path.join(loss_path, prefix+"val.txt"), "w") as f:
            for s in loss_val_list:
                f.write(str(s) +"\n")
            
    print("Finished", epoch + 1, "epochs of training")
print("Training complete after", epoch, "epochs")

tensor(0.2954, device='cuda:0')
Epoch 1, average minibatch 0 loss: 0.006
tensor(0.1799, device='cuda:0')
tensor(0.1984, device='cuda:0')
tensor(0.1864, device='cuda:0')
tensor(0.1377, device='cuda:0')
tensor(0.2258, device='cuda:0')
tensor(0.1300, device='cuda:0')
tensor(0.1970, device='cuda:0')
tensor(0.1630, device='cuda:0')
tensor(0.1341, device='cuda:0')
tensor(0.1173, device='cuda:0')
tensor(0.1742, device='cuda:0')
tensor(0.2779, device='cuda:0')
tensor(0.2768, device='cuda:0')
tensor(0.1791, device='cuda:0')
tensor(0.1775, device='cuda:0')
tensor(0.1559, device='cuda:0')
tensor(0.1462, device='cuda:0')
tensor(0.1591, device='cuda:0')
tensor(0.1755, device='cuda:0')
tensor(0.1743, device='cuda:0')
tensor(0.1709, device='cuda:0')
tensor(0.1405, device='cuda:0')
tensor(0.1398, device='cuda:0')
tensor(0.1761, device='cuda:0')
tensor(0.1863, device='cuda:0')
tensor(0.1724, device='cuda:0')
tensor(0.1380, device='cuda:0')
tensor(0.1251, device='cuda:0')
tensor(0.1849, device='cuda:0')

# inference

In [2]:
PATH = '/datasets/home/home-02/60/960/kshi/PA3/models/resnet_variant/20190214-162928/epoch_0-batch_700-loss_0.1824745088815689-20190214-173852.pt'
model_test = resnet_n(pretrained=False, num_classes=14)
model_test = model_test.to(computing_device)
model_test.load_state_dict(torch.load(PATH))

In [3]:
labels_all = []
predictions_all = []
for data in test_loader:
    images, labels = data
    
    images, labels = images.to(computing_device), labels.to(computing_device)
    labels_all.append(labels)
    output = model_best(images)
    predictions = output > 0.5
    predictions_all.append(predictions)

labels = torch.cat(labels_all,0)
predctions = torch.cat(predictions_all,0)

In [5]:
eval = Evaluation(predctions.float(), labels)
print(eval.accuracy())
print(eval.accuracy().mean())

TP [1654.0, 342.0, 894.0, 2438.0, 554.0, 969.0, 5.0, 474.0, 21.0, 203.0, 195.0, 135.0, 305.0, 2.0]
FP [15091.0, 13676.0, 7047.0, 11626.0, 10808.0, 16217.0, 498.0, 10112.0, 638.0, 9766.0, 8437.0, 10111.0, 11561.0, 2790.0]
TN [3064.0, 6002.0, 10723.0, 5019.0, 8372.0, 2820.0, 19446.0, 9168.0, 18714.0, 9988.0, 11291.0, 9792.0, 8055.0, 17360.0]
FN [372.0, 161.0, 1517.0, 1098.0, 447.0, 175.0, 232.0, 427.0, 808.0, 224.0, 258.0, 143.0, 260.0, 29.0]
tensor([0.2338, 0.3144, 0.5756, 0.3695, 0.4423, 0.1878, 0.9638, 0.4778, 0.9283,
        0.5050, 0.5691, 0.4919, 0.4143, 0.8603], device='cuda:0',
       dtype=torch.float64)
tensor(0.5238, device='cuda:0', dtype=torch.float64)


In [6]:
eval.precision()

tensor([0.0988, 0.0244, 0.1126, 0.1734, 0.0488, 0.0564, 0.0099, 0.0448, 0.0319,
        0.0204, 0.0226, 0.0132, 0.0257, 0.0007], device='cuda:0',
       dtype=torch.float64)

In [7]:
eval.recall()

tensor([0.8164, 0.6799, 0.3708, 0.6895, 0.5534, 0.8470, 0.0211, 0.5261, 0.0253,
        0.4754, 0.4305, 0.4856, 0.5398, 0.0645], device='cuda:0',
       dtype=torch.float64)